# Remove genomes

GCF_002158865.1 failed GTDBtk - it passed the initial busco filtering step but only had ~3% or so of the amino acids needed for gtdbtk. Not sure what the discrepancy is but it would be ideal to filter this out of all output files, including the mash centroid file (yuck).


Added `GCF_002158865.1` to `to_skip` variable in `stage_ncbi_assembly_summary.py`

removed .fna.gz file from `genomedb/ncbi-refseq-raw`

In [1]:
import os
import shutil
import subprocess
from Bio import SeqIO, Seq

In [2]:
genomes = ["GCF_002158865.1"]
folder = "/usr2/people/melnyk/genomedb/metadata"
os.mkdir("old")
os.mkdir("new")

In [3]:
for f in os.listdir(folder):
    print(f)
    shutil.copy(os.path.join(folder, f), "old")

genome_stats.csv
clusters.csv
centroids.msh
gtdbtk_taxonomy.tsv
gtdbtk_msa.faa


In [4]:
o = open("new/genome_stats.csv", 'w')
for line in open("old/genome_stats.csv"):
    vals = line.rstrip().split(",")
    if vals[0] not in genomes:
        o.write(line)
o.close()

In [5]:
o = open("new/clusters.csv", 'w')
for line in open("old/clusters.csv"):
    vals = line.rstrip().split(",")
    if vals[0] not in genomes:
        if vals[2] not in genomes:
            o.write(line)
o.close()

In [6]:
o = open("new/gtdbtk_taxonomy.tsv", 'w')
for line in open("old/gtdbtk_taxonomy.tsv", 'r'):
    vals = line.rstrip().split("\t")
    if vals[0].replace(".fna", "") in genomes:
        continue
    else:
        o.write(line)
o.close()

In [7]:
o = open("new/gtdbtk_msa.faa", 'w')
for seq in SeqIO.parse(open("old/gtdbtk_msa.faa", 'r'), 'fasta'):
    if seq.id.replace(".fna", "") not in genomes:
        SeqIO.write(seq, o, 'fasta')
o.close()

In [9]:
os.mkdir("fasta")

In [10]:
# create a folder "fasta" with the genome files for all centroid genomes
for line in open("new/clusters.csv", 'r'):
    if line.startswith("accession,"):
        continue
    vals = line.rstrip().split(",")
    if vals[1] == "True":
        cmds = [
            "ln", "-s",
            f"/usr2/people/melnyk/genomedb/ncbi-refseq-raw/{vals[0]}.fna.gz",
            "fasta"
        ]
        subprocess.run(cmds)

In [11]:
! mash info old/centroids.msh

/bin/bash: mash: command not found


Output from mash info old/centroids.msh
```
Header:
  K-mer size:                    21 (64-bit hashes)
  Alphabet:                      ACGT (canonical)
  Target min-hashes per sketch:  1000
  Sketches:                      27285
```

Ran the following commands to create a new centroid file:

```
cd fasta
parallel -j 8 gunzip -f ::: *.fna.gz
ls *.fna > fasta_list.txt
mash sketch \
  -p 8 \
  -l fasta_list.txt \
  -o ../new/centroids.msh
```

Output from mash info old/centroids.msh
```
Header:
  K-mer size:                    21 (64-bit hashes)
  Alphabet:                      ACGT (canonical)
  Target min-hashes per sketch:  1000
  Sketches:                      27284
```

Copy files back to ~/genomedb/metadata
```
 mv new/* ~/genomedb/metadata/
 ```